In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2M
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam

print("Available GPUs:", tf.config.list_physical_devices("GPU"))

Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# Path to your dataset directories
data_dir = "/kaggle/input/giloma-mdc-dataset/training/training"

# Data generators
train_datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # EfficientNetV2-M default input size
    batch_size=32,
    class_mode="binary",
    subset="training",
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    subset="validation",
)

Found 1241 images belonging to 2 classes.
Found 309 images belonging to 2 classes.


In [4]:
# Build the model
base_model = EfficientNetV2M(
    include_top=False, input_shape=(224, 224, 3), weights="imagenet"
)

model = Sequential(
    [base_model, GlobalAveragePooling2D(), Dense(1, activation="sigmoid")]
)

# model.summary()

214201816/214201816 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
# Compile the model with F1 score as a custom metric
def f1_score(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred)
    precision = tp / (tf.reduce_sum(y_pred) + 1e-7)
    recall = tp / (tf.reduce_sum(y_true) + 1e-7)
    return 2 * (precision * recall) / (precision + recall + 1e-7)


model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=[Precision(), Recall(), f1_score],
)

In [6]:
from tensorflow.keras.callbacks import EarlyStopping

# Add early stopping
early_stopping = EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    callbacks=[early_stopping],
    epochs=10,
)

# Save the model
model.save("binary_classifier_efficientnetv2m.h5")

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


39/39 ━━━━━━━━━━━━━━━━━━━━ 16s 394ms/step - f1_score: 13.7018 - loss: 0.0340 - precision: 0.9836 - recall: 0.9957 - val_f1_score: 0.0000e+00 - val_loss: 4.3028 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
